#  Class Activation Map이란 무엇인가?

## 2. Explainable AI
모델이 잘못된 답변을 준다면 어떻게 개선할 수 있을지, 잘 동작한다면 왜 이런 선택을 했는지 알고자 하는 것을 __XAI(explainable AI)__ 라고 한다.

### 이미지 분류 문제(Image Classification)
---
<img src="./image/img_classification.png" />
<center><b>https://3months.tistory.com/465</b></center>

위 그림처럼 분류모델에 이미지가 입력으로 들어온다고 생각해보자. 
* 일반적인 이미지 분류 모델은 이미지의 local feature를 추출하기 위해 CNN으로 구성된 특성 추출용 백본 네트워크(backbone network)가 앞에 있다. 백본 네트워크에서 추출된 특성 맵(feature map)을 fully connected layer에 통과시켜 얻어진 logit을 소프트맥스(softmax) 활성화 함수에 통과시키면, 입력 이미지가 각 클래스에 속할 확률을 얻을 수 있다.

logit은 아래와 같이 sigmoid의 역함수이며, softmax는 sigmoid를 K개의 클래스로 일반화한 것임을 알 수 있다.

<img src="./image/sigmoid.jpg" />

이런 이미지 분류 모델에서 얻어진 logit 값이 K개의 클래스 중 하나가 정답이라고 가리킨다면, 혹시 그렇게 분류할 수 있는 근거를 이미지 분류 모델 안에서 찾아볼 수도 있을 것 같다.

* 레이어마다 feature map을 시각화해서, activation이 어떻게 됬는지 확인해 볼 수 있을 것이다.

## 3. CAM: Class Activation Map
CAM은 Class Activation Map을 뜻한다. 단순화 시켜 이야기 하면 "모델이 어떤 곳을 보고 각 클래스임을 짐작하고 있는지" 확인할 수 있는 지도이다. CAM은 MIT 연구진의 _[Learning Deep Features for Discriminative Localization](http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf)_ 논문에서 확인할 수 있다.

CAM 논문의 연구진은 CAM을 얻기 위해서 GAP을 사용했으며, 이를 통해서 어떤 클래스가 어느 영역에 의해서 활성화되었는지 알 수 있도록 했다.

### 1) GAP(Global Average Pooling)
---
>__*주요 참고 자료*__
>
>* _[Network In Network](https://arxiv.org/abs/1312.4400)_
>* _[navisphere.net의 Network In Network리뷰](http://www.navisphere.net/5493/network-in-network/)_
>* _[C4W1L09 Pooling Layers](https://www.youtube.com/watch?v=8oOgPUO-TBY&feature=youtu.be)_

<img src="./image/gap.png" />
<center><b>[출처 : http://taewan.kim/post/cnn/]</b></center>

Pooling에는 average pooling과 max pooling 등이 있다.

* Average pooling은 커널과 겹치는 영역에 대해 Average 값을 취한 것.
* Max pooling은 커널과 겹치는 영역에 대해 Max 값을 취한 것.

<img src="./image/gap2.png" />
<center><b>https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/</b></center>

분류 모델의 마지막 부분에서 fully connected layer 대신 GAP을 사용하는 방법은 [Network in Network](https://arxiv.org/pdf/1312.4400.pdf)라는 논문에서 제안되었다.

* 일반적으로 이미지 분류를 위해서 여러 CNN 레이어를 거쳐 특성을 추출하고, 그 특성 맵을 flattening 한 후 fully connected layer에 입력해 줌으로써 각 클래스에 따른 logit을 구하고 최종 활성화 함수를 거친다.

GAP(Global Average Pooling)은 매 채널별로, 위에서 보았던 average pooling을 채널의 값 전체에 global하게 적용한다. 

_Network in Network_ 논문에서는 CNN 이후 fully connected layer를 사용하는 대신 GAP 연산을 한 후 이에 대해 소프트맥스 활성화 함수를 적용한다. 이때 마지막 CNN 레이어의 채널 수는 데이터의 클래스 수에 맞춰 각 클래스에 따른 확률을 얻을 수 있도록 했다. 원 논문에서는 이렇게 하면 특성 맵의 각 채널이 클래스별 신뢰도를 나타내게 되어 해석이 쉬울 뿐만 아니라, fully connected layer과 달리 최적화할 파라미터가 존재하지 않으므로 과적합(overfitting)을 방지할 수 있다고 설명한다.

<img src="./image/gap3.png" />
<center><b>https://arxiv.org/abs/1312.4400</b></center>

### 2) CAM
---
>__*주요 참고 자료*__
>
>* _[Learning Deep Features for Discriminative Localization](http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf)_
>* _[POD_Deep-LearningCAM - Class Activation Map의 CAM - Class Activation Map리뷰](https://poddeeplearning.readthedocs.io/ko/latest/CNN/CAM%20-%20Class%20Activation%20Map/)_

__CAM(Class Activation Map)__ 은 이름에서 보이듯이 클래스가 활성화되는 지도이다.

<img src="./image/cam.png" />
<center><b>http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf</b></center>

CAM을 얻을 수 있는 네트워크는 위 그림에서 볼 수 있다. 먼저 CNN 레이어를 거쳐 뽑아낸 특성 맵에 대해서 GAP을 적용한다. 그리고 이에 대해서 소프트맥스 레이어(소프트맥스 활성화 함수를 가지고 bias가 없는 fully connected layer)를 적용한다.

CNN을 거친 특성맵에서 각 클래스에 대한 정보는 결과값의 여러 채널에 걸쳐 나타나게 된다. GAP을 통해 각 채널 별 정보를 요약하면 소프트맥스 레이어는 이 정보를 보고 각 클래스에 대한 개별 채널의 중요도를 결정한다. 이렇게 클래스 별로 소프트맥스 레이어를 통해 각 채널의 가중합을 구하면 각 클래스가 활성화 맵의 어떤 부분을 주로 활성화 시키는지 확인할 수 있다.

우리가 이렇게 얻은 특성 맵은 CNN의 출력값 크기와 같게 된다. 이를 보간(interpolation)을 통해 적절히 원본 이미지 크기로 확대해 주면 위와 같은 CAM을 얻을 수 있다.